In [1]:
## subFunctions
import configparser
import os
from numpy import *
import cv2

def cfgRead():
    '''
    读取用于判定样品层数的相关阈值，从配置文件./OC_2D.cfg
    '''
    config = configparser.ConfigParser({'name_2D' : 'oc_ws2','waveLength' : '630'})
    config.read_file(open('./OC_2D.cfg'))

    # 读取存在的配置值，返回的是配置文件中的值
    name_2D = config.get('sample2inspect', 'name_2D')
    waveLength = config.get('sample2inspect', 'waveLength')

    sysDtec = name_2D+'_'+waveLength
    print('To detect '+sysDtec)
    if sysDtec == 'oc_default_470':
        ocVals =  config.options('oc_default_470')
    elif sysDtec == 'oc_ws2_630':
        ocVals =  config.options('oc_ws2_630')
    elif sysDtec == 'oc_ws2_530':
        ocVals =  config.options('oc_ws2_530')
    elif sysDtec == 'oc_ws2_470':
        ocVals =  config.options('oc_ws2_470')
    #elif 在配置文件中增加待检测项及对应的阈值时，这里增加对应的case语句，让ocVals能匹配对应参数
    #如果只增加一种待检测项，可以直接在default中修改阈值即可
        
    lenOc = len(ocVals)
    # print(lenOc); print(ocVals)
    if lenOc<4:
        print('Error! File OC_2D.cfg is not right.')
    else:
        thrSD = config.getfloat(sysDtec,ocVals[0])
        thrNL = zeros((lenOc-3))
        for i in range(lenOc-3):
            thrNL[i] = config.getfloat(sysDtec,ocVals[i+1])
    print('1-'+str(lenOc-3)+'L optical contrast:'); print(thrNL)
    print('Standard deviation:'); print(thrSD)
    return thrNL, thrSD, waveLength

def blpf(image,d):
    '''
    巴特沃斯低通滤波器
    '''
    f = fft.fft2(image)
    fshift = fft.fftshift(f)
    transfor_matrix = zeros(image.shape)
    M = transfor_matrix.shape[0]
    N = transfor_matrix.shape[1]
    for u in range(M):
        for v in range(N):
            D = sqrt((u-M/2)**2+(v-N/2)**2)
            transfor_matrix[u,v]=1/(1+pow(D/d,16))
    new_img = abs(fft.ifft2(fft.ifftshift(fshift * transfor_matrix)))
    return new_img

def rofDenoise(im,U_init,tolerance=0.1,tau=0.125,tv_weight=100):
    """ 使用 A. Chambolle（2005）在公式（11）中的计算步骤实现 Rudin-Osher-Fatemi（ROF）去噪模型
    输入：含有噪声的输入图像（灰度图像）、 U 的初始值、 TV 正则项权值、步长、停业条件
    输出：去噪和去除纹理后的图像、纹理残留 """
    m,n = im.shape # 噪声图像的大小
    # 初始化
    U = U_init
    Px = im # 对偶域的x 分量
    Py = im # 对偶域的y 分量
    error = 1
    while (error > tolerance):
        Uold = U
        # 原始变量的梯度
        GradUx = roll(U,-1,axis=1)-U # 变量 U 梯度的x 分量
        GradUy = roll(U,-1,axis=0)-U # 变量 U 梯度的y 分量
        # 更新对偶变量
        PxNew = Px + (tau/tv_weight)*GradUx
        PyNew = Py + (tau/tv_weight)*GradUy
        NormNew = maximum(1,sqrt(PxNew**2+PyNew**2))
        Px = PxNew/NormNew # 更新x 分量（对偶）
        Py = PyNew/NormNew # 更新y 分量（对偶）
        # 更新原始变量
        RxPx = roll(Px,1,axis=1) # 对x 分量进行向右x 轴平移
        RyPy = roll(Py,1,axis=0) # 对y 分量进行向右y 轴平移
        DivP = (Px-RxPx)+(Py-RyPy) # 对偶域的散度
        U = im + tv_weight*DivP # 更新原始变量
        # 更新误差
        error = linalg.norm(U-Uold)/sqrt(n*m);
    return U,im-U # 去噪后的图像和纹理残余

def histeq(im,nbr_bins=256):
    """ 对一幅灰度图像进行直方图均衡化 """
    # 计算图像的直方图
    imhist,bins = histogram(im.flatten(),nbr_bins,normed=True)
    cdf = imhist.cumsum() # cumulative distribution function
    cdf = 255 * cdf / cdf[-1] # 归一化
    # 使用累积分布函数的线性插值，计算新的像素值
    im2 = interp(im.flatten(),bins[:-1],cdf)
    return im2.reshape(im.shape), cdf

def ifConn(M):
    '''判断一个3*3矩阵是否从中间分割成两半,已知M的中间为1'''    
    b=reshape(M,(9))
    c=[b[0],b[1],b[2],b[5],b[8],b[7],b[6],b[3]]
    times=0
    for i in range(8):
        j=0 if i==7 else i+1
        if c[i]!=c[j]:
            times+=1
    if times>2:
        return True
    else:
        return False

def addConn(M, N,mode):
    '''给一个3*3边缘矩阵M添加边缘认知,已知M的中间为1;N为参考矩阵'''
    b=reshape(M,(9))
    c=[b[0],b[1],b[2],b[5],b[8],b[7],b[6],b[3]]
    d=reshape(N,(9))
    e=[d[0],d[1],d[2],d[5],d[8],d[7],d[6],d[3]]
    dIJ=[[-1,-1],[-1,0],[-1,1],[0,1],[1,1],[1,0],[1,-1],[0,-1]]
#     print('All: M,c,N')#     print(M)#     print(c)#     print(N)
    for i in range(8):
        j=0 if i==7 else (i+1)
        if (c[i]!=c[j]):
            e[i]=0;e[j]=0
        if (c[i]==1):
            e[i]=0
    idx=argmax(e)
    if e[idx]<0.1:  #待调参数，需要配合参考矩阵N的整体均值或中值（我们的场景是N为信息熵，均值约3，中值约0.05）
        di=7;dj=7
    else:
        di,dj=dIJ[idx]
    ifZero=1
    
    #边界需要很不一样的处理
    if mode==0:
        jdx=0 if idx==7 else idx+1
        kdx=7 if idx==0 else idx-1
    
        if M[dIJ[jdx][0],dIJ[jdx][1]]==1:
            di,dj=dIJ[jdx]
            ifZero=0
        elif M[dIJ[kdx][0],dIJ[kdx][1]]==1:
            di,dj=dIJ[kdx]
            ifZero=0
        else:
            ifZero=1
    
    return di,dj,ifZero

def homofilter(I,rL=0.2,rH=2,d0=100,c=0.1):
    '''灰度图像，同态滤波'''
    m,n = I.shape 
    I1 = np.log(I+1)
    FI = np.fft.fft2(I1)
    FI = np.fft.fftshift(FI)
    n1 = np.floor(m/2) 
    n2 = np.floor(n/2) 
    D = np.zeros((m,n)) 
    H = np.zeros((m,n)) 
    for i in range(m): 
        for j in range(n): 
            D[i,j]=((i-n1)**2+(j-n2)**2)
            H[i,j]=(rH-rL)*(1-np.exp(c*(-D[i,j]/(2*d0**2))))+rL
    I2 = np.fft.ifft2(np.fft.fftshift(H*FI))
    I3 = np.real(np.exp(I2)-1)
    # 指数处理ge = exp(g)-1;  # 归一化到[0, L-1]
    Imin = I3.min()
    Range=I3.max()- Imin
    for i in range(m):
        for j in range(n):
            I3[i, j] = uint8(255 * (I3[i, j] - Imin) / Range);
    return I3

In [4]:
## 191125 开始 综合版 的进一步完善。关注图像区域的分割优化
## 191219 完成图像分割和识别整理如下

##-------自定义函数 （tool函数在单独文件imTools.py,定义subFunctions）------
# subFuns in cell 2

##-------模块儿载入，程序运行环境搭建-------
%matplotlib qt5
import matplotlib.pyplot as plt
from imageio import imwrite
from matplotlib import image
from PIL import Image
from pylab import *
from numpy import *
from scipy.ndimage import filters
from scipy.ndimage import measurements
from scipy.ndimage import morphology as mpho
from skimage import data,color,morphology,feature,transform
import skimage.filters.rank as sfr

## 添加中文字体支持
from matplotlib.font_manager import FontProperties
font = FontProperties(fname=r"c:\windows\fonts\SimSun.ttc", size=14)
# print(ss)

## 参数列表（标准样品测量+统计 决定）
thrNL, thrSD, waveLength = cfgRead()
r,g,b = 0, 1, 2
if  waveLength==470:
    chn=b
elif waveLength==530:
    chn=g
else:
    chn=r

##--------打开背景图片，滤波--------
imPath = './PicOrigin/'  # 0927
tmpPath = './PicOutput/'

# im_BG0 = array(Image.open(imPath + '530bg1.bmp'))
# im_BG = double(im_BG0[:,:,chn]) #

# im0 =  blpf(im_BG,40)
# im0 = filters.gaussian_filter(im0,sigma=16)

##--------打开待处理图片，先后进行同态滤波、rof降噪
imFName = '03p10-abL-w7x325-530-111.bmp'#'04p21-23L-w7x300-530.bmp'   #03p10-abL-w7x325-530-111.bmp
im_IN0 = array(Image.open(imPath + imFName)) #  x530-2.bmp
im_IN = double(im_IN0[:,:,1])  #
# im_IN = double(im_IN0[350:1250,650:1550,1])  #
# im_IN = double(im_IN0[677:1061,1411:1505,1])  #
im1 = homofilter(im_IN,rL=0.2,rH=2,d0=900,c=0.1)
# im1 = im_IN
imwrite(tmpPath +'origin_'+imFName,im_IN) #保存为灰度图
imwrite(tmpPath +'afHomoFilter_'+imFName,im1)

U,T = rofDenoise(im1,im1)
##--------计算(R-R0)/R0，optical contrast
avgBG = 35; #没有背景照片时，认为是均匀背景，根据im1的背景部分手动设置背景强度avgBG,04p21对应28,03p10对应35
im2 = (double(im_IN) - avgBG)/avgBG
# im2 = (im1 - im0)/im0 #有可信的背景照片时

a=im2>0;
im2=im2*a; #im2(im2<0)=0截断 (也许转换成uint8时自动正确截断)

image.imsave(tmpPath + 'ocImg.tiff', uint8(im2/im2.max()*250)) #直接存导致奇怪的图, imshow却可以
imwrite(tmpPath +'afROF_'+imFName, uint8(U))

##---------计算信息熵imEn
imag = im2/im2.max()*250+1
dist =sfr.entropy(uint8(imag), morphology.disk(9)) #看起来熵图粒度能反应区分样品和杂质区域
# figure();sc = plt.imshow(dist, cmap = plt.cm.jet)# 设置cmap为RGB图
# plt.colorbar()# 显示色度条
imEn = transform.rescale(dist, 0.5) #缩小（~边缘锐化）
imEn[0,:]=0;imEn[-1,:]=0;imEn[:,0]=0;imEn[:,-1]=0
matplotlib.image.imsave(tmpPath + 'Entropy.tiff', uint8(imEn/imEn.max()*250)) #归一化到0-250

##----------为了canny边缘检测，再次滤波，并通过缩小图像去降低计算量、缩窄边缘
im1 = blpf(im1,64) #去燥同时尽量保边缘
im1 = filters.gaussian_filter(U,sigma=3)
im4 = transform.rescale(im1, 0.5) #缩小（~边缘锐化）
imwrite(tmpPath +'afBLPF_'+imFName, im4)

# edges1 = feature.canny(im4, sigma=1 , low_threshold=5, high_threshold=15)#good after blpf+rof
edges2 = feature.canny(im4, sigma=2 , low_threshold=4, high_threshold=12)#good after homoFilter+blpf+rof
imwrite(tmpPath +'canny_'+imFName, uint8(edges2*255))

##----------图像边缘和内部，分别进行边缘连接迭代   （目标：imEn和edges2联合分割图像成块儿！）
#-----首先修补图像边界
tx=imEn>0.5
thrEn =imEn.mean()#外边第2像素根据前景背景分割阈值置一
print('thrEn',thrEn)
LtIJ=list()
M,N=shape(imEn)
for i in [1,M-2]:#标记端点
    Lt=list()
    Lt.append([i,1])#首
    for j in range(1,N-1):
        encoderT=imEn[(i-1):(i+2),(j-1):(j+2)]*1
        if (imEn[i,j]==encoderT.max()) and (imEn[i,j]>thrEn):
            Lt.append([i,j])
            print([i,j])
        if (edges2[i,j]==1):
            Lt.append([i,j])
    Lt.append([i,N-2])#尾
    LtIJ.append(Lt)
print(LtIJ)
for j in [1,N-2]:
    Lt=list()
    Lt.append([1,j])#首
    for i in range(1,M-1):
        encoderT=imEn[(i-1):(i+2),(j-1):(j+2)]*1
        if (imEn[i,j]==encoderT.max()) and (imEn[i,j]>thrEn):
            Lt.append([i,j])
            print([i,j])
        if (edges2[i,j]==1):
            Lt.append([i,j])
    Lt.append([M-2,j])#尾
    LtIJ.append(Lt)
for ij in range(4):#通过端点连接边缘区物体
    for k in range(len(LtIJ[ij])-1):
        idx1=LtIJ[ij][k];idx2=LtIJ[ij][k+1]
        print(idx1,idx2)
        print('thrEnNow',imEn[idx1[0]:idx2[0]+1,idx1[1]:idx2[1]+1].mean())
        if imEn[idx1[0]:idx2[0]+1,idx1[1]:idx2[1]+1].mean()>thrEn:
            edges2[idx1[0]:idx2[0]+1,idx1[1]:idx2[1]+1]=1
edges2[[0,-1],:]=0;edges2[:,[0,-1]]=0;#外边1像素置零           

print('GOOD?')
#-----搜索内部边缘检测之断点
listA=list();
edgeEnd = edges2*50;
encoderM = array([[9,2,9],[2,1,2],[9,2,9]])
encoderT = zeros((3,3))
for i in range(1,M-1):
    for j in range(1,N-1):
        if edges2[i,j]==1:
            encoderT=edges2[(i-1):(i+2),(j-1):(j+2)]*1
            x=encoderT.sum()
            if x==1:
                edgeEnd[i,j]=0
                edges2[i,j]=0
            elif x==2:
                edgeEnd[i,j]=250
                listA.append([i,j])
            elif not ifConn(encoderT):
                encoderT=edges2[(i-1):(i+2),(j-1):(j+2)]*encoderM
                if encoderT.sum()==12:
                    edgeEnd[i,j]=250
                    listA.append([i,j])
                else:
                    edgeEnd[i,j]=150
matplotlib.image.imsave(tmpPath +'edgeEnds.tiff', uint8(edgeEnd))
print('Edges end at:',listA)

#-----生长连接内部边缘检测断点
for ij in range(len(listA)):
    idx=listA[ij]
    i=idx[0];j=idx[1]
    
    if i==1 or i==M-2 or j==1 or j==N-2:
        mode=0 #"图片边界"
    else:
        mode=1 #"图片内部"
    
    encoderM=edges2[(i-1):(i+2),(j-1):(j+2)]
    encoderT=imEn[(i-1):(i+2),(j-1):(j+2)]
    
    deltaI,deltaJ,ifZero=addConn(encoderM, encoderT,mode)
#     print(encoderM)
#     print([i,j,deltaI,deltaJ,ifZero])
    if deltaI>1:
        continue
    
    edges2[i+deltaI,j+deltaJ]=1
    edges2[i,j]=ifZero
    encoderM=edges2[(i+deltaI-1):(i+deltaI+2),(j+deltaJ-1):(j+deltaJ+2)]
    encoderT = imEn[(i+deltaI-1):(i+deltaI+2),(j+deltaJ-1):(j+deltaJ+2)]
    i=i+deltaI;j=j+deltaJ
    if i<2 or i>M-2 or j<2 or j>N-2:
        continue
    while(not ifConn(encoderM)):
        if i==1 or i==M-2 or j==1 or j==N-2:
            mode=0 #"图片边界"
        else:
            mode=1 #"图片内部"

        deltaI,deltaJ,ifZero=addConn(encoderM, encoderT, mode)
        if deltaI>1:break
        edges2[i+deltaI,j+deltaJ]=1
        edges2[i,j]=ifZero
        encoderM=edges2[(i+deltaI-1):(i+deltaI+2),(j+deltaJ-1):(j+deltaJ+2)]
        encoderT = imEn[(i+deltaI-1):(i+deltaI+2),(j+deltaJ-1):(j+deltaJ+2)]
        i=i+deltaI;j=j+deltaJ
#         print(encoderM)
#         print([i,j,deltaI,deltaJ,ifZero])
        
#         matplotlib.image.imsave(tmpPath +'edges4.tiff', uint8(edges2)) #查看出错时的状态
        if i<2 or i>M-2 or j<2 or j>N-2:
            break            
imwrite(tmpPath +'cnntdCanny_'+imFName, uint8(edges2*255))

##----------根据边缘二值图，进行形态学处理，获得分块儿物体
edges3=morphology.closing(edges2,morphology.disk(3))
edges3[[0,-1],:]=0;edges3[:,[0,-1]]=0;#外边1像素置零

chull = mpho.binary_fill_holes(edges3)
imOBJnot=morphology.dilation(edges3)
matplotlib.image.imsave(tmpPath +'OBJs0.tiff', chull)
matplotlib.image.imsave(tmpPath +'OBJs2.tiff', chull&~imOBJnot)

#------分块儿标记
imLast = transform.rescale(chull&~imOBJnot, 2)
labels_open, nbr_objects_open = measurements.label(imLast)
imwrite(tmpPath +'asampleParts0.tiff', uint8(labels_open*10))

labels_open1=zeros(labels_open.shape)
thrArea = 1000 #可调参数，物体最小面积
Areas=list()
iNL=len(thrNL)
dthrNL = ones(iNL+2)*thrSD
dthrNL[1:-2] = thrNL[1:] - thrNL[0:-1]
for i in range(nbr_objects_open):
    a=(labels_open == (i)).sum()
    Areas.append(a)
#     print('i+a:',i,a)
    if a > thrArea:
        b = im2[labels_open == (i)].sum()/a   #mean optical contrast
#         print('b=',b)
        mark=0 #初始标记
        if b<(thrNL[0]-dthrNL[0]/2):#判断是否背景
            print('bg',i,a,b)
            mark=1 #标记识别成功
            labels_open1[labels_open==(i)] = 0 
        else:
            for li in range(iNL):#判断是否薄层
                if (b>(thrNL[li]-dthrNL[li]/2)) and (b<(thrNL[li]+dthrNL[li+1]/2)):
                    print('few',i,li+1,a,b)
                    mark=1 #标记识别成功
                    labels_open1[labels_open==(i)] = li*30+30
        if (mark==0):#其他情况认为是厚层
#             print('thick',i,a,b)
            labels_open1[labels_open==(i)] = 250

    elif a > thrArea/10:#较小的图像块儿判断层数无效
#         print('Not',i,a)
        labels_open1[labels_open==(i)] = 250
    else:#过小的图像块儿忽略到背景
        labels_open1[labels_open==(i)] = 0

##-----------上色
im_color = cv2.applyColorMap(uint8(labels_open1), cv2.COLORMAP_SPRING)
im_color[labels_open1==(0),:]=0
im_color[labels_open1==(250),:]=250
imwrite(tmpPath +'asampleParts1.tiff', uint8(labels_open1))
imwrite(tmpPath +'asampleParts2.tiff', im_color)

#-----同步保存colorBar
A=np.multiply(30*np.array([[1],[2],[3],[4],[5]]),np.ones((1,50)))
B=np.ones((1,100))
cBar0 = np.multiply(np.reshape(A,(250,1)),B)
cBar = cv2.applyColorMap(uint8(cBar0), cv2.COLORMAP_SPRING)
imwrite(tmpPath +'colorBar.tiff', cBar)

#file ends here

To detect oc_ws2_530
1-5L optical contrast:
[0.21866    0.3977175  0.7839225  1.16170333 1.4385925 ]
Standard deviation:
0.130807896


C:\ProgramData\Anaconda3\lib\site-packages\skimage\transform\_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "


thrEn 1.4639915735150735
[1, 16]
[1, 23]
[1, 26]
[[[1, 1], [1, 16], [1, 23], [1, 26], [1, 62], [1, 398]], [[498, 1], [498, 47], [498, 62], [498, 70], [498, 398]]]
[25, 1]
[51, 1]
[56, 1]
[105, 1]
[285, 1]
[288, 1]
[1, 1] [1, 16]
thrEnNow 3.2236069187910172
[1, 16] [1, 23]
thrEnNow 3.3475811041841923
[1, 23] [1, 26]
thrEnNow 3.394270045167526
[1, 26] [1, 62]
thrEnNow 3.5636750684803657
[1, 62] [1, 398]
thrEnNow 0.03869511436672663
[498, 1] [498, 47]
thrEnNow 0.45099293607713753
[498, 47] [498, 62]
thrEnNow 4.801527900866074
[498, 62] [498, 70]
thrEnNow 5.062804367444526
[498, 70] [498, 398]
thrEnNow 0.12653864414941027
[1, 1] [25, 1]
thrEnNow 3.2368677239786177
[25, 1] [51, 1]
thrEnNow 3.163432813085259
[51, 1] [56, 1]
thrEnNow 3.28835860824355
[56, 1] [105, 1]
thrEnNow 3.5045429992264268
[105, 1] [108, 1]
thrEnNow 4.8581554069660005
[108, 1] [154, 1]
thrEnNow 0.5519893480843697
[154, 1] [171, 1]
thrEnNow 4.302837086801959
[171, 1] [275, 1]
thrEnNow 0.14520511082369758
[275, 1] [276, 1]